In [1]:
!kaggle datasets download -d mathurinache/math-dataset
!unzip /content/math-dataset.zip

from IPython.display import clear_output
clear_output()

In [3]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers==0.0.27" trl peft accelerate bitsandbytes

In [4]:
from unsloth import FastLanguageModel
import torch


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [26]:
prompt = """
### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # end of sentence token
def formatting_prompts(sample):
    problems = sample["problem"]
    solutions       = sample["solution"]
    texts = []
    for problem, solution in zip(problems, solutions):
        text = prompt.format(problem, solution) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


In [22]:
from datasets import Dataset
from pathlib import Path
import json

def generator():
  ds_dir   = "/content/MATH/train"
  dir_path = Path(ds_dir)

  for i in list(dir_path.rglob("*.json")): yield json.load(open(i, "r"))

####


In [27]:
from datasets import load_dataset

dataset = Dataset.from_generator(generator)
dataset = dataset.map(formatting_prompts, batched = True,)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [28]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

{'problem': 'Given that 0.5 liters is approximately 1.05 pints, how many pints are in one liter? Express your answer as a decimal to the nearest tenth.',
 'level': 'Level 3',
 'type': 'Prealgebra',
 'solution': 'If half a liter is 1.05 pints, then a whole liter is $2\\cdot1.05=\\boxed{2.1}$ pints.',
 'text': '\n### Question:\nGiven that 0.5 liters is approximately 1.05 pints, how many pints are in one liter? Express your answer as a decimal to the nearest tenth.\n\n### Response:\nIf half a liter is 1.05 pints, then a whole liter is $2\\cdot1.05=\\boxed{2.1}$ pints.<|end_of_text|>'}